In [1]:
import time
import twitter
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

import dateutil.parser
import json

class Tweet(dict):
    def __init__(self, tweet_in):
        super(Tweet, self).__init__(self)
        if tweet_in and 'delete' not in tweet_in:
            self['timestamp'] = dateutil.parser.parse(tweet_in[u'created_at']
                                ).replace(tzinfo=None).isoformat()
            self['text'] = tweet_in['text'].encode('utf-8')
            self['hashtags'] = [x['text'].encode('utf-8') for x in tweet_in['entities']['hashtags']]
            self['geo'] = tweet_in['geo']['coordinates'] if tweet_in['geo'] else None
            self['id'] = tweet_in['id']
            self['screen_name'] = tweet_in['user']['screen_name'].encode('utf-8')
            self['user_id'] = tweet_in['user']['id'] 
        
def connect_twitter():
    twitter_stream =  twitter.TwitterStream(auth=twitter.OAuth(token = "107968834-rWAKtQNiTiGDRRC5W7Iy3JDgag6SH5rbpvJoxYQP",token_secret = "BXBt4mG9LUJoWKPiua8rBoSD7XM2PMEc9byDygK4bXwS0",consumer_key = "wKC0XeeDsF2Nq6r443dsIrTE7",consumer_secret = "r1GcpLLTtbEqRcZE0x8W7MD0vucfklk6nPxEec7hIpIdoO3F3j"))
    return twitter_stream

def get_next_tweet(twitter_stream):
    stream = twitter_stream.statuses.sample(block=True)
    tweet_in = None
    while not tweet_in or 'delete' in tweet_in:
        tweet_in = stream.next()
        tweet_parsed = Tweet(tweet_in)
        if not "delete" in tweet_in:
            print "-- " , tweet_parsed['screen_name'], " # " , tweet_parsed['timestamp'], " # " , tweet_parsed['text']
        # print(json.dumps(tweet_in, indent=2, sort_keys=True))
    # return json.dumps(tweet_in, indent=2, sort_keys=True)
    return json.dumps(tweet_parsed)

def process_rdd_queue(twitter_stream):
    rddQueue = []
    for i in range(5):
        rddQueue += [ssc.sparkContext.parallelize([get_next_tweet(twitter_stream)], 5)]

    lines = ssc.queueStream(rddQueue)
    lines.saveAsTextFiles("out_files/files_")
    lines.pprint()
    

In [2]:
twitter_stream = connect_twitter()

In [3]:
ssc = StreamingContext(sc, 1)
process_rdd_queue(twitter_stream)
ssc.start()
time.sleep(4)
ssc.stop(stopSparkContext=False, stopGraceFully=True)
time.sleep(100)
ssc = StreamingContext(sc, 1)
process_rdd_queue(twitter_stream)
ssc.start()
time.sleep(4)
ssc.stop(stopSparkContext=False, stopGraceFully=True)

--  natsumikisa7016  #  2016-04-13T11:01:17  #  仕事で近くに行ってたんだけど
すごかった！！！！👓の壁！ https://t.co/ZpSEqqZ2Hy
--  holvoeteline  #  2016-04-13T11:01:19  #  RT @EmmelieVanhool: Ik mis men mooiste sterretje hierboven 😔😇❤️
--  sultan_ssdd  #  2016-04-13T11:01:20  #  RT @AliKerrouche: إذا أردت التوقف عن القلق والبدء بالحياة .. عدد نعمك وليس متاعبك..#قروب_غيم_للدعم
--  AmazingInfoTech  #  2016-04-13T11:01:21  #  RT @cloudera: Video Demo: Getting Started with Hive-on-Spark https://t.co/VaDB6kV4fC
--  bukkket_  #  2016-04-13T11:01:22  #  RT @garez_ank: #VeBenimYalanım ÖLMÜS  OLAN DEDEM BİRDAHA  ÖLDÜ BUGÜN İŞE GELEMEYECEM
-------------------------------------------
Time: 2016-04-13 13:01:23
-------------------------------------------
{"user_id": 217079960, "screen_name": "natsumikisa7016", "timestamp": "2016-04-13T11:01:17", "hashtags": [], "text": "\u4ed5\u4e8b\u3067\u8fd1\u304f\u306b\u884c\u3063\u3066\u305f\u3093\u3060\u3051\u3069\n\u3059\u3054\u304b\u3063\u305f\uff01\uff01\uff01\uff01\ud83d\udc53\u30

KeyboardInterrupt: 